<a href="https://colab.research.google.com/github/etgins/Mice_ASD_Detection/blob/main/audio_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

----------------------------------------------
Written by Itamar Ginsberg & Alon Schreuer, October 2021



# **1. Import data and split labels**

---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
import numpy as np

# import dataset from xls
import pandas as pd

## COLAB WORK
## ITAMAR:
#dataset = pd.read_excel('/content/drive/MyDrive/Project_A/Project_A_files/total_data UPDATED EM 020821.xlsx')
## ALON:
dataset = pd.read_csv("/content/drive/MyDrive/total_data.csv")
## ITAMAR:
#dataset.to_csv (r'/content/drive/MyDrive/Project_A/Project_A_files/total_data UPDATED EM 020821.csv', index = None, header=True)
# dataset.dropna(axis = 0, how='any')

## ALON:
#dataset.to_csv("/content/drive/MyDrive/total_data.csv", index = None, header=True)
#dataset = pd.read_csv("/content/drive/MyDrive/total_data.csv", error_bad_lines=False)

# extract only the relevant columns / features
X = dataset[["Name", "Start Point (Hz)", "End Point (Hz)", "Duration (s)", "Syllable number"]]
y = dataset[["Offspring Genotype"]]

# TODO - clean NaN values from dataset or X
X.dropna(axis = 0, how='any')
X

# TODO - convert duration to seconds from 'datetime.time' type to 'timedelta' object


## TODO - encode text feature values
# from sklearn.preprocessing import LabelEncoder
# l1 = LabelEncoder()
# l1.fit(X['Name'])
# dataset.Name = l1.transform(dataset.Name)
# dataset

# X.describe()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Name,Start Point (Hz),End Point (Hz),Duration (s),Syllable number
0,17470O,NaN,NaN,NaN,NaN
1,17470O,57648.8278,45525.0820,00:00:00.059000,3.0
2,17470O,55558.5268,45525.0820,00:00:00.063000,2.0
3,17470O,54513.3763,43852.8412,00:00:00.063000,2.0
4,17470O,55976.5870,43434.7810,00:00:00.060000,2.0
...,...,...,...,...,...
4363,17472I,NaN,NaN,NaN,7.0
4364,17472I,NaN,NaN,NaN,7.0
4365,17472I,NaN,NaN,NaN,7.0
4366,17472I,NaN,NaN,NaN,1.0


# **2. Split data for different mice by name - unfinished**

---

In [ ]:
# split dataset for each mouse

def take_data_by_name(name_searched, Data, name_list):
  # print(name_list)
  ind = name_list == name_searched
  # print(ind)
  matching_data = Data[ind]
  # print(matching_data)
  return matching_data


# find set of mouse names
name_column = dataset["Name"]
unique_name_set = set(name_column)
unique_name_list = list(unique_name_set)
# unique_name_list = unique_name_list[0:-1]   # first entry is column name
number_of_mice = len(unique_name_list)
print(number_of_mice, 'mice in data')
unique_name_list.pop(0)
print(unique_name_list)


51 mice in data
['08106M', '08101L', '17455J', '17450L', '08102I', '08027I', '17444K', '17474J', '08057P', '17453I', '08130I', '17471P', '08103J', '08060M', '17490J', '17484J', '08136G', '17489I', '08028L', '17477I', '08056J', '17470O', '08114J', '17479L ', '08139L', '08119O', '08112K', '08104K', '17482K', '17456K', '08105P', '08111I', '17454L', '08142K', '08110N', '08134N(2)', '08139L(2)', '08059O', '17479L', '08109B', '08096P', '17472I', '08169M', '17491K', '08113G', '08132N', '08121P', '08107O', '08137B', '08098I']


# **3. Extract features**

---

In [1]:
## --------------------------------------------------------
# TODO - for each mouse, take only data that belongs to it :
# note: not yet finished, for now use the data for all mice in X
## --------------------------------------------------------

## 1. example - for one mouse only:
## ------------------------------
# idx = 0
# mouse_split_data = take_data_by_name(unique_name_list[idx], X, name_column)
# print('mouse name:', unique_name_list[idx],', ' 'idx:', idx)
# mouse_split_data

## 2. trying for all mice:
## ---------------------
  # final data is made up of: 
    # avg. start freq. per syllable
    # avg. end freq. per syllable
    # avg. syllable duration per syllable
    # syllable distribution - percentage of each syllable
    # Bandwidth - TODO

num_of_syllables = 10
final_data_size = 4*num_of_syllables
# initialize finalized data variable: mouse_final_data
mouse_final_data = np.empty([number_of_mice,final_data_size])


for idx in range (0,1): #(number_of_mice):
  mouse_split_data = take_data_by_name(unique_name_list[idx], X, name_column)
# print(mouse_split_data)
#  print('mouse name:', unique_name_list[idx],', ' 'idx:', idx)
#  print('matching data:', '\n',mouse_split_data)

  # calculate feature 1 - average start freq for each syllable (Alon)
  ## -------------------------------------------

  # TODO 
  # extract all syllables per mouse - done
  # create a dictionay with num of syllable as key and all it's start freqs arrange in list as a value
  # for every value in the dictionary compute the average
  # to it again in a loop for every mouse such that in the end I'll get an array with num of mouse as a key and the value is the dictionay with the average of start freqs per syllable (*think about better way to arrange the data)
  

  # create a dictionay with num of syllable as key and all it's start freqs arrange in list as a value
  dict = {}
  start_freq = np.array(mouse_split_data["Start Point (Hz)"])
  syllable_num = np.array(mouse_split_data["Syllable number"])
  count = 0
  for i in syllable_num:
    if i == None:
      count += 1
      print(count)
    dict[i] = start_freq[i]
  
  print(start_freq)
  print(len(start_freq))

  
  

  # get the num of each syllable 


  # calculate feature 2 - average end freq for each syllable (Alon)
  ## -------------------------------------------


  # calculate feature 3 - syllable distribution (Itamar):
  ## -------------------------------------------
# get rid of NaN
  mouse_split_data = mouse_split_data.dropna(subset=["Syllable number"])

# take syll numbers
  syllable_num_row = np.transpose(np.array(mouse_split_data["Syllable number"]))
  # print(syllable_num_row)

# initialize distribution vector
  distribution = np.zeros([1,10])
  # print(distribution)

# increment distribution count
  for i in range(len(syllable_num_row)):
    # print("iteration", i)
    syll = int(syllable_num_row[i])
    # print("found syllable", syll)
    # print("old distribution:", distribution)
    distribution[0,syll-1] += 1 
    # print("new distribution:", distribution)

# normalize histogram to distribution
  distribution = distribution / distribution.sum()
  # print("final distribution:", distribution)

# transfer results to mouse_final_data
  mouse_final_data[idx,21:31] = distribution


  # calculate feature 4 - average syllable duration for each syllable (Alon)
  ## -------------------------------------------

  a = np.copy(mouse_split_data.groupby('Syllable number'))
  # mouse_split_data = mouse_split_data.astype({"Duration (s)" : int})
#  for syllable_idx in range(9):
    # print("syllable idx is:", syllable_idx)
    # print("split duration col is:")
    # print(a[syllable_idx](0))
    # mean = np.mean(a[syllable_idx]())
    # print(syllable_idx, "syllable duration mean is:", mean)

 # print('this is a:', a[0])
  
  # calculate feature 5 - Bandwidth (???) - TODO
  ## -------------------------------------------

#  print(mouse_final_data[idx,:])
  

NameError: ignored